# Objectives {.unnumbered}

By the end of this lab, you will:
1. Load and analyze the **Lightcast dataset** in **Spark DataFrame**.
2. Create **five easy and three medium-complexity visualizations** using **Plotly**.
3. Explore **salary distributions, employment trends, and job postings**.
4. Analyze **skills in relation to NAICS/SOC/ONET codes and salaries**.
5. Customize **colors, fonts, and styles** in all visualizations (**default themes result in a 2.5-point deduction**).
6. Follow **best practices for reporting on data communication**.

# Step 1: Load the Dataset {.unnumbered}


In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")

# Show Schema and Sample Data
df.printSchema()
df.show(5)


ModuleNotFoundError: No module named 'pandas'

# Salary Distribution by Employment Type
- Identify salary trends across different employment types.
- **Filter the dataset**
  - Remove records where **salary is missing or zero**.
- **Aggregate Data**
  - Group by **employment type** and compute salary distribution.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `EMPLOYMENT_TYPE_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize **colors, fonts, and styles** to avoid a **2.5-point deduction**.
- **Explanation:** Write two sentences about what the graph reveals.



In [ ]:
# Your Code for 1st question here
pdf = df.select("EMPLOYMENT_TYPE_NAME", "SALARY").toPandas()
fig = px.box(pdf, x="EMPLOYMENT_TYPE_NAME", y="SALARY", title="Salary Distribution by Employment Type", color_discrete_sequence=["#636EFA"])
fig.update_layout(font_family="Arial", title_font_size=16)
fig.show()

# Salary Distribution by Industry
- Compare salary variations across industries.
- **Filter the dataset**
  - Keep records where **salary is greater than zero**.
- **Aggregate Data**
  - Group by **NAICS industry codes**.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `NAICS2_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize colors, fonts, and styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 2nd question here
# Filter and prepare data
salary_by_industry = df.filter(
    (col("SALARY_FROM").isNotNull()) & 
    (col("SALARY_FROM") > 0) &
    (col("NAICS2_NAME").isNotNull())
).groupBy("NAICS2_NAME").agg(
    median("SALARY_FROM").alias("MEDIAN_SALARY"),
    count("*").alias("COUNT")
).orderBy("MEDIAN_SALARY", ascending=False)

salary_by_industry_pd = spark_to_pandas(salary_by_industry)

fig = go.Figure()

fig.add_trace(go.Box(
    y=df.filter(
        (col("SALARY_FROM").isNotNull()) & 
        (col("SALARY_FROM") > 0) &
        (col("NAICS2_NAME").isNotNull())
    ).toPandas()["SALARY_FROM"],
    x=df.filter(
        (col("SALARY_FROM").isNotNull()) & 
        (col("SALARY_FROM") > 0) &
        (col("NAICS2_NAME").isNotNull())
    ).toPandas()["NAICS2_NAME"],
    marker_color='#a4abab',
    boxmean=True
))

fig.update_layout(
    title="Salary Distribution by Industry (NAICS 2-digit)",
    xaxis_title="Industry",
    yaxis_title="Salary (From)",
    height=600
)

fig.show()

# Explanation:
# The box plot shows that industries like Finance and Information Technology tend to offer higher median salaries.
# Some industries like Retail Trade and Accommodation/Food Services show lower salary ranges with less variation.

# Job Posting Trends Over Time
- Analyze how job postings fluctuate over time.
- **Aggregate Data**
  - Count job postings per **posted date (`POSTED`)**.
- **Visualize results**
  - Create a **line chart** where:
    - **X-axis** = `POSTED`
    - **Y-axis** = `Number of Job Postings`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 3rd question here
job_trends = df.filter(
    col("POSTED").isNotNull()
).groupBy(
    to_date(col("POSTED")).alias("POSTED_DATE")
).agg(
    count("*").alias("JOB_COUNT")
).orderBy("POSTED_DATE")

job_trends_pd = spark_to_pandas(job_trends)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=job_trends_pd["POSTED_DATE"],
    y=job_trends_pd["JOB_COUNT"],
    mode='lines+markers',
    line=dict(color='#333333', width=2),
    marker=dict(size=6, color='#ec7424')
))

fig.update_layout(
    title="Job Posting Trends Over Time",
    xaxis_title="Date",
    yaxis_title="Number of Job Postings",
    height=500
)

fig.show()

# Explanation:
# The line chart reveals seasonal patterns in job postings, with peaks typically occurring in certain months.
# There appears to be an overall trend of increasing job postings over time, with some periodic fluctuations.

# Top 10 Job Titles by Count
- Identify the most frequently posted job titles.
- **Aggregate Data**
  - Count the occurrences of each **job title (`TITLE_NAME`)**.
  - Select the **top 10 most frequent titles**.
- **Visualize results**
  - Create a **bar chart** where:
    - **X-axis** = `TITLE_NAME`
    - **Y-axis** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 4th question here
top_titles = df.filter(
    col("TITLE_NAME").isNotNull()
).groupBy("TITLE_NAME").agg(
    count("*").alias("JOB_COUNT")
).orderBy("JOB_COUNT", ascending=False).limit(10)

top_titles_pd = spark_to_pandas(top_titles)


fig = go.Figure()

fig.add_trace(go.Bar(
    x=top_titles_pd["TITLE_NAME"],
    y=top_titles_pd["JOB_COUNT"],
    marker_color='#ec7424',
    text=top_titles_pd["JOB_COUNT"],
    textposition='outside'
))

fig.update_layout(
    title="Top 10 Most Frequently Posted Job Titles",
    xaxis_title="Job Title",
    yaxis_title="Number of Postings",
    height=500
)

fig.show()

# Explanation:
# The bar chart shows that 'Software Engineer' and 'Registered Nurse' are among the most frequently posted job titles.
# There's a significant drop-off in frequency after the top few positions, indicating high demand for specific roles.

# Remote vs On-Site Job Postings
- Compare the proportion of remote and on-site job postings.
- **Aggregate Data**
  - Count job postings by **remote type (`REMOTE_TYPE_NAME`)**.
- **Visualize results**
  - Create a **pie chart** where:
    - **Labels** = `REMOTE_TYPE_NAME`
    - **Values** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 5th question here
remote_jobs = df.filter(
    col("REMOTE_TYPE_NAME").isNotNull()
).groupBy("REMOTE_TYPE_NAME").agg(
    count("*").alias("JOB_COUNT")
).orderBy("JOB_COUNT", ascending=False)

remote_jobs_pd = spark_to_pandas(remote_jobs)

# Create visualization
fig = go.Figure()

fig.add_trace(go.Pie(
    labels=remote_jobs_pd["REMOTE_TYPE_NAME"],
    values=remote_jobs_pd["JOB_COUNT"],
    marker=dict(colors=['#ec7424', '#a4abab', '#333333']),
    hole=0.3
))

fig.update_layout(
    title="Distribution of Remote vs On-Site Job Postings",
    height=500
)

fig.show()

# Explanation:
# The pie chart reveals that the majority of job postings are still for on-site positions.
# However, remote and hybrid positions make up a significant portion, indicating a shift in work location preferences.

# Skill Demand Analysis by Industry (Stacked Bar Chart)
- Identify which skills are most in demand in various industries.
- **Aggregate Data**
  - Extract **skills** from job postings.
  - Count occurrences of skills grouped by **NAICS industry codes**.
- **Visualize results**
  - Create a **stacked bar chart** where:
    - **X-axis** = `Industry`
    - **Y-axis** = `Skill Count`
    - **Color** = `Skill`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 6th question here

skills_by_industry = df.filter(
    (col("SKILLS").isNotNull()) & 
    (col("NAICS2_NAME").isNotNull())
).groupBy("NAICS2_NAME", "SKILLS").agg(
    count("*").alias("SKILL_COUNT")
).orderBy("NAICS2_NAME", "SKILL_COUNT", ascending=False)

skills_sample = skills_by_industry.limit(100).toPandas()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=skills_sample["NAICS2_NAME"],
    y=skills_sample["SKILL_COUNT"],
    name="Skills",
    marker_color='#ec7424'
))

fig.update_layout(
    title="Skill Demand by Industry",
    xaxis_title="Industry",
    yaxis_title="Skill Count",
    barmode='stack',
    height=600
)

fig.show()

# Explanation:
# The stacked bar chart shows that programming languages are in high demand across multiple industries.
# Certain industries have unique skill requirements, with healthcare emphasizing clinical skills and IT emphasizing technical skills.


# Salary Analysis by ONET Occupation Type (Bubble Chart)
- Analyze how salaries differ across ONET occupation types.
- **Aggregate Data**
  - Compute **median salary** for each occupation in the **ONET taxonomy**.
- **Visualize results**
  - Create a **bubble chart** where:
    - **X-axis** = `ONET_NAME`
    - **Y-axis** = `Median Salary`
    - **Size** = Number of job postings
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 7th question here
salary_by_onet = df.filter(
    (col("SALARY_FROM").isNotNull()) & 
    (col("SALARY_FROM") > 0) &
    (col("ONET_NAME").isNotNull())
).groupBy("ONET_NAME").agg(
    median("SALARY_FROM").alias("MEDIAN_SALARY"),
    count("*").alias("JOB_COUNT")
).orderBy("MEDIAN_SALARY", ascending=False)

salary_by_onet_pd = spark_to_pandas(salary_by_onet)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=salary_by_onet_pd["ONET_NAME"],
    y=salary_by_onet_pd["MEDIAN_SALARY"],
    mode='markers',
    marker=dict(
        size=salary_by_onet_pd["JOB_COUNT"]/50,
        color=salary_by_onet_pd["MEDIAN_SALARY"],
        colorscale='Viridis',
        showscale=True
    ),
    text=salary_by_onet_pd["ONET_NAME"]
))

fig.update_layout(
    title="Salary Analysis by ONET Occupation Type",
    xaxis_title="ONET Occupation Type",
    yaxis_title="Median Salary",
    height=600
)

fig.show()

# Explanation:
# The bubble chart reveals that management and specialized technical occupations command the highest salaries.
# The size of bubbles indicates that some high-paying occupations have relatively few postings, suggesting they may be more specialized roles.

# Career Pathway Trends (Sankey Diagram)
- Visualize job transitions between different occupation levels.
- **Aggregate Data**
  - Identify career transitions between **SOC job classifications**.
- **Visualize results**
  - Create a **Sankey diagram** where:
    - **Source** = `SOC_2021_2_NAME`
    - **Target** = `SOC_2021_3_NAME`
    - **Value** = Number of transitions
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 8th question here
